In [10]:
import os
import sys
import pandas as pd
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertConfig

sys.path.insert(0, '../')
from config import Config
from load_data import load_data

In [ ]:
def tokenized_dataset(dataset, tokenizer):
    concat_entity = []
    for e01, e02 in zip(dataset["entity_01"], dataset["entity_02"]):
        temp = ""
        temp = e01 + "[SEP]" + e02
        concat_entity.append(temp)
    tokenized_sentences = tokenizer(
        concat_entity,
        list(dataset["sentence"]),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=100,
        add_special_tokens=True,
    )
    return tokenized_sentences

In [11]:
tokenizer = AutoTokenizer.from_pretrained(Config.BERTMultiLingual)

In [13]:
train_raw = load_data(Config.Train)

In [17]:
concat_entity = []
for e01, e02 in zip(train_raw["entity_01"], train_raw["entity_02"]):
    temp = ""
    temp = e01 + "[SEP]" + e02
    concat_entity.append(temp)
tokenized_sentences = tokenizer(
    concat_entity,
    list(train_raw["sentence"]),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=100,
    add_special_tokens=True,
)

In [62]:
temp = tokenizer(text=concat_entity, text_pair=list(train_raw['sentence']))
len(temp['input_ids']), len(temp['input_ids'][0])

(9000, 66)

In [ ]:
class REDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_dataset, labels):
        self.tokenized_dataset = tokenized_dataset
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(val[idx]) for key, val in self.tokenized_dataset.items()
        }
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
